In [37]:
import requests
import json
import pandas as pd

GRAPH_URL_PREFIX = 'https://graph.facebook.com/'
FIELDS_CONJ = '?fields='
LIMITS_CONJ- '?limit='
SUMMARY_REQUEST='?summary=true'

COMMUNITY_FIELDS='groups'

GROUPS_SUFFIX = '/groups'
GROUP_FIELDS = 'id,name,members,privacy,description,updated_time'

MEMBERS_SUFFIX = '/members'
MEMBER_FIELDS = 'email,id,administrator,name'

FEED_SUFFIX= '/feed'
FEED_FIELDS='id,created_time,updated_time,from,message'
#Adjust how many results you want to stay visible
FEED_LIMIT=1000

POST_FIELDS= 'id,created_time,updated_time,from,message'

ACCESS_TOKEN= 'DQVJ2LTNQa3B0M3dra1hfdTJnT2YyNmw2X0cxUHJlMWFtWW9XWW9pbDdSVkJYYkNqMlFfb0RidTU4anRXM3E5LTU2MHFsZA2hnSGZAYUEhPOHBTTGpfYlk3UUh4S2xzX1VMeF9VNkk0cjFvd09hRFlpR21DcERqUlRaYjZAIT3BfcFY1Ry1OVWJsZAkdzS21mSG1xVkZAtb1FSbFU0UDZAUM0wwWTE2TlBpY3VpSmQxd2M5eFlSWmVScnBxbmVIYl9QZAGZASNUw1bDhR'

JSON_KEY_DATA = 'data'
JSON_KEY_PAGING = 'paging'
JSON_KEY_NEXT = 'next'
JSON_KEY_EMAIL = 'email'

#Group IDS
TT_MAIN="284578622020844"

def getGroupMembers(access_token, group_id):
    endpoint = GRAPH_URL_PREFIX + group_id + MEMBERS_SUFFIX + FIELDS_CONJ + MEMBER_FIELDS
    return getPagedData(access_token, endpoint, [])

def getPostFields(access_token, post_id):
    endpoint= GRAPH_URL_PREFIX + post_id+ FIELDS_CONJ+ POST_FIELDS
    print(endpoint)
    return getJsonData(access_token, endpoint)

def getPostSummary(access_token, post_id, summary_type):
    endpoint= GRAPH_URL_PREFIX + post_id +"/" +summary_type +SUMMARY_REQUEST
    print(endpoint)
    return getJsonData(access_token, endpoint)

def getCommunityNumber(access_token):
    endpoint= GRAPH_URL_PREFIX+'community'
    return getJsonData(access_token, endpoint)

def getCommunityGroups(access_token, community_ID):
    endpoint= GRAPH_URL_PREFIX +community_ID+ FIELDS_CONJ + COMMUNITY_FIELDS
    print(endpoint)
    return getJsonData(access_token, endpoint)



def getGroupFeed(access_token, group_id):
    endpoint = GRAPH_URL_PREFIX + group_id + FEED_SUFFIX+ FIELDS_CONJ+ FEED_FIELDS 
    return getJsonData(access_token, endpoint)

def getPagedData(access_token, endpoint, data):
    headers = buildHeader(access_token)
    result = requests.get(endpoint,headers=headers)
    result_json = json.loads(result.text)
    json_keys = result_json.keys()
    if JSON_KEY_DATA in json_keys and len(result_json[JSON_KEY_DATA]):
        data.extend(result_json[JSON_KEY_DATA])
    if JSON_KEY_PAGING in json_keys and JSON_KEY_NEXT in result_json[JSON_KEY_PAGING]:
        next = result_json[JSON_KEY_PAGING][JSON_KEY_NEXT]
        if next:
            getPagedData(access_token, next, data)
    return data

def getJsonData(access_token, endpoint):
    headers = buildHeader(access_token)
    result = requests.get(endpoint,headers=headers)
    result_json = json.loads(result.text)
    return result_json
    

def buildHeader(access_token):
    return {'Authorization': 'Bearer ' + access_token}

In [38]:
#First, we get all the groups for this community getCommunityNumber(ACCESS_TOKEN)

{'name': 'Quest Alliance', 'privacy': 'CLOSED', 'id': '112511455959143'}

In [50]:
#Now get all the groups for the community, then the posts in each group and finally the comments for each post
raw_groups= getCommunityGroups(ACCESS_TOKEN,'112511455959143')
all_groups=[]
for group in raw_groups['groups']['data']:
    all_groups.append(group)
    group['feed']=getGroupFeed(ACCESS_TOKEN, group['id'])
    #for post in group['feed']['data']:
        #post['comments']=getPostSummary(ACCESS_TOKEN, post['id'],'comments')
    
with open('GroupwiseData.json', 'w') as outfile:
        json.dump(all_groups, outfile, indent=4)

 
#print(comments)
    #for post in group['feed']['data']
print(all_groups)


    

https://graph.facebook.com/112511455959143?fields=groups
[{'name': 'Chat bot testing', 'privacy': 'SECRET', 'id': '143184776370656', 'feed': {'data': [{'id': '143184776370656_163644884324645', 'created_time': '2018-04-21T10:35:42+0000', 'updated_time': '2018-04-23T08:47:20+0000', 'from': {'name': 'Neha', 'id': '100018752347018'}, 'message': 'testing to write in multiple languages'}, {'id': '143184776370656_143184946370639', 'created_time': '2018-01-23T05:00:23+0000', 'updated_time': '2018-04-08T08:47:28+0000', 'from': {'name': 'Gandhali Sewak', 'id': '100018673056852'}, 'message': 'This group is for testing the chat bot integration/features etc. Limited people only.'}, {'id': '143184776370656_143213809701086', 'created_time': '2018-01-23T08:48:46+0000', 'updated_time': '2018-01-23T08:48:46+0000', 'from': {'name': 'Gandhali Sewak', 'id': '100018673056852'}, 'message': 'As of now we have created 2-3 bots. To check, in your message window (where you type answer) a bot icon will come, clic

In [51]:
#Next, we will repeat this action in a separate file for likes
    
for group in all_groups:
    for post in group['feed']['data']:
        post['comments']= getPostSummary(ACCESS_TOKEN, post['id'],'comments')

with open('GroupWiseCommentsData.json', 'w') as outfile:
        json.dump(all_groups, outfile, indent=4)

https://graph.facebook.com/143184776370656_163644884324645/comments?summary=true
https://graph.facebook.com/143184776370656_143184946370639/comments?summary=true
https://graph.facebook.com/143184776370656_143213809701086/comments?summary=true
https://graph.facebook.com/143184776370656_143184786370655/comments?summary=true
https://graph.facebook.com/132870324079980_153619365338409/comments?summary=true
https://graph.facebook.com/132870324079980_149796359054043/comments?summary=true
https://graph.facebook.com/132870324079980_132871177413228/comments?summary=true
https://graph.facebook.com/132870324079980_132870330746646/comments?summary=true
https://graph.facebook.com/975280852635501_1013587605471492/comments?summary=true
https://graph.facebook.com/975280852635501_1013601335470119/comments?summary=true
https://graph.facebook.com/975280852635501_1013714582125461/comments?summary=true
https://graph.facebook.com/975280852635501_1013591595471093/comments?summary=true
https://graph.facebook.c

https://graph.facebook.com/148774729037763_182398219008747/comments?summary=true
https://graph.facebook.com/148774729037763_196547780927124/comments?summary=true
https://graph.facebook.com/148774729037763_196213004293935/comments?summary=true
https://graph.facebook.com/148774729037763_195909874324248/comments?summary=true
https://graph.facebook.com/148774729037763_155401468375089/comments?summary=true
https://graph.facebook.com/148774729037763_195177004397535/comments?summary=true
https://graph.facebook.com/148774729037763_193633887885180/comments?summary=true
https://graph.facebook.com/148774729037763_172613683320534/comments?summary=true
https://graph.facebook.com/148774729037763_165200370728532/comments?summary=true
https://graph.facebook.com/148774729037763_180372175878018/comments?summary=true
https://graph.facebook.com/112376982763245_171077200226556/comments?summary=true
https://graph.facebook.com/112376982763245_161228677878075/comments?summary=true
https://graph.facebook.com/1

https://graph.facebook.com/284578622020844_401535030325202/comments?summary=true
https://graph.facebook.com/284578622020844_400340430444662/comments?summary=true
https://graph.facebook.com/284578622020844_400477970430908/comments?summary=true
https://graph.facebook.com/284578622020844_400060973805941/comments?summary=true
https://graph.facebook.com/284578622020844_399051273906911/comments?summary=true
https://graph.facebook.com/284578622020844_400279580450747/comments?summary=true
https://graph.facebook.com/284578622020844_400246820454023/comments?summary=true
https://graph.facebook.com/284578622020844_400242850454420/comments?summary=true
https://graph.facebook.com/284578622020844_400237913788247/comments?summary=true
https://graph.facebook.com/112373509463067_115204842513267/comments?summary=true
https://graph.facebook.com/112373509463067_113684542665297/comments?summary=true
https://graph.facebook.com/112373509463067_112564979443920/comments?summary=true
https://graph.facebook.com/1

In [53]:
for group in all_groups:
    for post in group['feed']['data']:
        post['reactions']= getPostSummary(ACCESS_TOKEN, post['id'],'reactions')

with open('GroupWiseReactionsData.json', 'w') as outfile:
        json.dump(all_groups, outfile, indent=4)

https://graph.facebook.com/143184776370656_163644884324645/reactions?summary=true
https://graph.facebook.com/143184776370656_143184946370639/reactions?summary=true
https://graph.facebook.com/143184776370656_143213809701086/reactions?summary=true
https://graph.facebook.com/143184776370656_143184786370655/reactions?summary=true
https://graph.facebook.com/132870324079980_153619365338409/reactions?summary=true
https://graph.facebook.com/132870324079980_149796359054043/reactions?summary=true
https://graph.facebook.com/132870324079980_132871177413228/reactions?summary=true
https://graph.facebook.com/132870324079980_132870330746646/reactions?summary=true
https://graph.facebook.com/975280852635501_1013587605471492/reactions?summary=true
https://graph.facebook.com/975280852635501_1013601335470119/reactions?summary=true
https://graph.facebook.com/975280852635501_1013714582125461/reactions?summary=true
https://graph.facebook.com/975280852635501_1013591595471093/reactions?summary=true
https://grap

https://graph.facebook.com/148774729037763_199884683926767/reactions?summary=true
https://graph.facebook.com/148774729037763_199240087324560/reactions?summary=true
https://graph.facebook.com/148774729037763_182398219008747/reactions?summary=true
https://graph.facebook.com/148774729037763_196547780927124/reactions?summary=true
https://graph.facebook.com/148774729037763_196213004293935/reactions?summary=true
https://graph.facebook.com/148774729037763_195909874324248/reactions?summary=true
https://graph.facebook.com/148774729037763_155401468375089/reactions?summary=true
https://graph.facebook.com/148774729037763_195177004397535/reactions?summary=true
https://graph.facebook.com/148774729037763_193633887885180/reactions?summary=true
https://graph.facebook.com/148774729037763_172613683320534/reactions?summary=true
https://graph.facebook.com/148774729037763_165200370728532/reactions?summary=true
https://graph.facebook.com/148774729037763_180372175878018/reactions?summary=true
https://graph.fa

https://graph.facebook.com/284578622020844_395695617575810/reactions?summary=true
https://graph.facebook.com/284578622020844_401944956950876/reactions?summary=true
https://graph.facebook.com/284578622020844_401527556992616/reactions?summary=true
https://graph.facebook.com/284578622020844_401535030325202/reactions?summary=true
https://graph.facebook.com/284578622020844_400340430444662/reactions?summary=true
https://graph.facebook.com/284578622020844_400477970430908/reactions?summary=true
https://graph.facebook.com/284578622020844_400060973805941/reactions?summary=true
https://graph.facebook.com/284578622020844_399051273906911/reactions?summary=true
https://graph.facebook.com/284578622020844_400279580450747/reactions?summary=true
https://graph.facebook.com/284578622020844_400246820454023/reactions?summary=true
https://graph.facebook.com/284578622020844_400242850454420/reactions?summary=true
https://graph.facebook.com/284578622020844_400237913788247/reactions?summary=true
https://graph.fa

In [7]:
TEST_POST='284578622020844_399075683904470'
TT_MAIN="284578622020844"

#r=requests.get('https://graph.facebook.com/284578622020844_401535030325202?fields=id,created_time,updated_time,from,message')
#r.headers
getPostFields(ACCESS_TOKEN, TEST_POST)

https://graph.facebook.com/284578622020844_399075683904470?fields=id,created_time,updated_time,from,message


{'id': '284578622020844_399075683904470',
 'created_time': '2018-05-15T12:24:58+0000',
 'updated_time': '2018-05-23T11:03:57+0000',
 'from': {'name': 'Pallavi Mishra', 'id': '100024692932936'},
 'message': 'Today conducted a guest session on Health, Nutrition & Menstruation Cycle for "girls student of GOVT ITI, Ajmer.  One of the main objective  of  this session was "to create an understanding about health and awareness  about menstruation cycle."\n\n**Topics were-: **\n\n1.Health,its importance and factors affecting health\n\n2. Nutrition, deficiency of nutrition and Balanced diet\n\n3. An important topic was also covered with the context of women/girls, that was about  the **anemia, menstruation cycle, hygiene during periods and myths about periods in our society. ** \n\n**Feedback from students-: ** Before  this session I tried to talk about this with girls and I got to know that they are unaware about the menstruation cycle, why it happen with them/only with girls. That time I also

In [17]:

feeds= getGroupFeed(ACCESS_TOKEN, TT_MAIN)
print(len(feeds['data']))
#feeds=feeds[0:400]
print(feeds)
#all_reactions=[]
#for feed in feeds_sample:
    #post_id=feed['id']
    #all_reactions.append(getPostSummary(ACCESS_TOKEN, post_id,'reactions'))
#len(all_reactions)

#feeds= json.load(feeds)
#df=pd.read_json(feeds)
#print(df)

all_posts=[]
for post in feeds['data']:
    all_posts.append(post)
    
        


25
{'data': [{'id': '284578622020844_402361126909259', 'created_time': '2018-05-23T08:19:32+0000', 'updated_time': '2018-05-24T07:42:16+0000', 'from': {'name': 'Trainer Tribe', 'id': '100021479812612'}, 'message': '# Take a look at these movie posters. All of these movies had an important role in them - A Teacher/ Mentor / Coach. \n# With respect to this role, which of these movies did you like the most? And why?'}, {'id': '284578622020844_402691733542865', 'created_time': '2018-05-24T04:21:39+0000', 'updated_time': '2018-05-24T04:21:39+0000', 'from': {'name': 'SRIDHAR.V', 'id': '100021877244863'}, 'message': '1. How to learn these SKILLS in  Trainer Tribe \n2. What you say about this ?'}, {'id': '284578622020844_400209747124397', 'created_time': '2018-05-18T04:39:58+0000', 'updated_time': '2018-05-23T16:53:50+0000', 'from': {'name': 'Neha', 'id': '100018752347018'}, 'message': "# HOW DO YOU GREET YOUR STUDENTS EVERYDAY? \n\nDid you ever think about the first time you meet your student

In [9]:
all_posts=[]
for post in feeds['data']:
    all_posts.append(post)
print(all_posts)

[{'id': '284578622020844_402361126909259', 'created_time': '2018-05-23T08:19:32+0000', 'updated_time': '2018-05-24T07:42:16+0000', 'from': {'name': 'Trainer Tribe', 'id': '100021479812612'}, 'message': '# Take a look at these movie posters. All of these movies had an important role in them - A Teacher/ Mentor / Coach. \n# With respect to this role, which of these movies did you like the most? And why?'}, {'id': '284578622020844_402691733542865', 'created_time': '2018-05-24T04:21:39+0000', 'updated_time': '2018-05-24T04:21:39+0000', 'from': {'name': 'SRIDHAR.V', 'id': '100021877244863'}, 'message': '1. How to learn these SKILLS in  Trainer Tribe \n2. What you say about this ?'}, {'id': '284578622020844_400209747124397', 'created_time': '2018-05-18T04:39:58+0000', 'updated_time': '2018-05-23T16:53:50+0000', 'from': {'name': 'Neha', 'id': '100018752347018'}, 'message': "# HOW DO YOU GREET YOUR STUDENTS EVERYDAY? \n\nDid you ever think about the first time you meet your students in the day

In [16]:
for post in all_posts:
    post['comments']= getPostSummary(ACCESS_TOKEN, post['id'],'comments')

with open('CommentsData.json', 'w') as outfile:
        json.dump(all_posts, outfile, indent=4)
      
    
#print(comments)

In [7]:
with open('CommentData.json', 'w') as outfile:
        json.dump(all_reactions, outfile, indent=4)

In [ ]:
for a in feeds

In [17]:
for feed in feeds:
    post_id=feed['id'] 
    feed['comments']=getPostSummary(ACCESS_TOKEN, post_id,'comments')

with open('CommentsData.json', 'w') as outfile:
        json.dump(feeds, outfile, indent=4)
        

In [24]:
all_comments=[]
m=0
while m<2:
    for feed in feeds_sample:
        post_id=feed['id']
        post_message= feed['message']
        print(post_id, post_message)
        m+=1
                
        
        
    #all_comments.append(getPostSummary(ACCESS_TOKEN, post_id,'comments'))
#len(all_comments)

with open('CommentsData.json', 'w') as outfile:
        json.dump(all_comments, outfile, indent=4)
        

284578622020844_399075683904470 Today conducted a guest session on Health, Nutrition & Menstruation Cycle for "girls student of GOVT ITI, Ajmer.  One of the main objective  of  this session was "to create an understanding about health and awareness  about menstruation cycle."

**Topics were-: **

1.Health,its importance and factors affecting health

2. Nutrition, deficiency of nutrition and Balanced diet

3. An important topic was also covered with the context of women/girls, that was about  the **anemia, menstruation cycle, hygiene during periods and myths about periods in our society. ** 

**Feedback from students-: ** Before  this session I tried to talk about this with girls and I got to know that they are unaware about the menstruation cycle, why it happen with them/only with girls. That time I also found hesitation in them. some of them started smiling.   

After the session girls said that they got to know about their  body, importance and process of menstruation cycle, hygiene 

KeyError: 'message'

In [8]:
#Group IDS
all_users=getGroupMembers(ACCESS_TOKEN, TT_MAIN)

with open('allMembers.json', 'w') as outfile:
        json.dump(all_users, outfile)
        


In [ ]:
graph_url = GRAPH_URL_PREFIX + "community" + GROUPS_SUFFIX + params
result = requests.get(graph_url, headers=headers)
result_json = json.loads(result.text, result.encoding)


"graph.facebook.com/{post-id}/comments?summary=true"

In [3]:
for feed in feeds:
    post_id=feed['id']
    all_comments= getPostSummary(ACCESS_TOKEN, post_id,'comments')
with open('testCommentData.json', 'w') as outfile:
        json.dump(all_comments, outfile)

SSLError: HTTPSConnectionPool(host='graph.facebook.com', port=443): Max retries exceeded with url: /284578622020844_384210295391009/comments?summary=true (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')",),))

In [3]:
with open('testItemData.json', 'w') as outfile:
        json.dump(feeds, outfile)


In [14]:
getPostSummary(ACCESS_TOKEN, '284578622020844_395695617575810','reactions' )

[{'id': '100018833221253', 'name': 'Pallavi Rao', 'type': 'LIKE'},
 {'id': '100021982639863', 'name': 'Mr.Bamdev Tripathi', 'type': 'LIKE'},
 {'id': '100018752347018', 'name': 'Neha', 'type': 'LIKE'},
 {'id': '100022032163939', 'name': 'Vivek Shihori', 'type': 'LIKE'}]

In [9]:
test_post='284578622020844_395695617575810'
endpoint= GRAPH_URL_PREFIX + post_id+

r= request.get(GRAPH_URL_PREFIX + post_id + summary_type
def getPostSummary(access_token, post_id, summary_type):
    endpoint= GRAPH_URL_PREFIX + post_id + summary_type+SUMMARY_REQUEST
    return getPagedData(access_token, endpoint, [])

[]